In [1]:
from pulp import *

# 整数線形計画の基本事項

* 整数線形計画（Integer Linear Programming; ILP）：
    * 目的関数と制約式が線形
    * 変数の一部または全部が整数
* ILPとして定式化できる$\neq$整数計画ソルバーで解ける．以下の場合は注意が必要
    * 変数や制約式の数が膨大になる場合
    * 定式化に非常に大きい数（big-M）が含まれる場合  
    -> big-Mで表現せず元の表現の特徴を使った解法を適用するのが妥当なこともある（制約プログラミングとの融合はこのアプローチ）
* 一般に，次の条件を満たす定式化ほど良い定式化（整数計画ソルバーによって解きやすい定式化）である
    1. 線形計画緩和が良い（凸包が良い）
    2. 変数の数が少ない
    3. 制約式の数が少ない
    4. （極端に大きい・小さい係数を含めない）  
    
  例えば，big-Mを含む定式化は係数が大きいうえに(1)の意味で弱いことが多く，多用しないことを心掛けるべきであるとされている．また，変数の上下限が予めわかっていれば，それを記述しておくのが良い

ILP1

In [14]:
# 問題の定義
prob = LpProblem(name='ILP1', sense=LpMaximize)

# 変数
x1 = LpVariable('x1', lowBound=0.0, cat='Integer')
x2 = LpVariable('x2', lowBound=0.0, cat='Integer')

# 定式化
prob += 4*x1 + 5*x2
prob += 2*x1 + 2*x2 <= 7
prob += 3*x1 + 5*x2 <= 14

print(prob)  # 問題
prob.solve()  # 求解
# 結果を表示
print('Status: ', LpStatus[prob.status])
print(f'Optimal value = {value(prob.objective)}')
for v in prob.variables():
    print(f'{v}={value(v)}')

ILP1:
MAXIMIZE
4*x1 + 5*x2 + 0
SUBJECT TO
_C1: 2 x1 + 2 x2 <= 7

_C2: 3 x1 + 5 x2 <= 14

VARIABLES
0 <= x1 Integer
0 <= x2 Integer

Status:  Optimal
Optimal value = 14.0
x1=1.0
x2=2.0


実行可能解は，$(x1, x2) = (0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (3, 0)$が実行可能解であるから，以下の定式化も同じ意味の定式化

In [22]:
prob = LpProblem(name="ILP1'", sense=LpMaximize)
x1 = LpVariable('x1', lowBound=0.0, cat='Integer')
x2 = LpVariable('x2', lowBound=0.0, cat='Integer')

prob += 4*x1 + 5*x2
prob += x1 + x2 <= 3
prob += x2 <= 2

print(prob)
prob.solve()
print('Status: ', LpStatus[prob.status])
print(f'Optimal value = {value(prob.objective)}')
for v in prob.variables():
    print(f'{v}={value(v)}')

ILP1':
MAXIMIZE
4*x1 + 5*x2 + 0
SUBJECT TO
_C1: x1 + x2 <= 3

_C2: x2 <= 2

VARIABLES
0 <= x1 Integer
0 <= x2 Integer

Status:  Optimal
Optimal value = 14.0
x1=1.0
x2=2.0


* このように線形制約が異なると，線形計画緩和問題も異なる
* 例えば，ILP1'の線形計画緩和問題の実行可能領域（多面体）はILP1のそれに含まれている．前者はILPの実行可能集合の凸包であり，これ以上領域を小さくするように線形制約を記述することはできない
* つまり，凸法は最強の定式化（理想の定式化）を与えるが，それを記述する線形制約をすべて求めることは一般に非常に困難である．ただし，最近の整数計画ソルバーは凸包に近づける機能（前処理，カット生成など）を備えている．例えば，ILP1の$2x_1+2x_2\leq 7$は$x_1+x_2\leq 3.5$と同値だが，整数条件によって$x_1+x_2\leq 3$とすることができる（前処理のひとつ）
* 凸包：集合$S$を含む凸集合のなかで（包含関係の意味で）最小のものを凸包と呼ぶ．$S$が有限集合の場合，凸包は多面体，つまり線形不等式系で記述できることが知られている（Weylの定理）

ILP2

In [21]:
NPV = [17, 16, 14, 10, 8]
budget = [60, 50, 40, 30, 20]

prob = LpProblem(name='ILP2', sense=LpMaximize)
x = [LpVariable(f'x{i}', lowBound=0, cat='Binary') for i in range(len(NPV))]

prob += lpDot(NPV, x)
prob += lpDot(budget, x) <= 100

print(prob)
prob.solve()
print('Status: ', LpStatus[prob.status])
print(f'Optimal value = {value(prob.objective)}')
for v in prob.variables():
    print(f'{v}={value(v)}')

ILP2:
MAXIMIZE
17*x0 + 16*x1 + 14*x2 + 10*x3 + 8*x4 + 0
SUBJECT TO
_C1: 60 x0 + 50 x1 + 40 x2 + 30 x3 + 20 x4 <= 100

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer

Status:  Optimal
Optimal value = 34.0
x0=0.0
x1=1.0
x2=0.0
x3=1.0
x4=1.0


# 様々な表現方法